**Inclass material for Week 4: SQL Query using `pandas`**

This notebook was made based on main materials `4_SQL_Query.ipynb`

Version: January 2022


---
**START OF DAY 1**

# SQL Query using `pandas`

**Training Objectives**

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statements
- Flavors and Common Operators
- End to end data analysis

# Working with SQL Databases

## Database Schema

Database `chinook.db` terdiri dari 11 tabel yang berisi sampel data dari sebuah toko yang menjual media digital:

- Data terkait media seperti pada tabel `tracks`, `albums`, `artists`, `genres`, `media_types`, `playlists`, `playlist_track` merupakan data asli dari library Apple iTunes.
- Informasi mengenai `customers` dan `employees` dibuat menggunakan nama dan alamat fiktif yang dapat ditemukan di Google maps, dan data lain yang diformat dengan baik (telepon, fax, email, dll).
- Informasi penjualan `invoices` dan `invoice_items` dihasilkan secara otomatis menggunakan data acak untuk periode tahun 2009-2013.

Skema berikut sering disebut sebagai **Entity Relationship Diagram (ERD)**, menunjukkan:
1. Entitas (Tabel)
2. Atribut (Kolom beserta tipe datanya)
3. Kardinalitas (Hubungan antar tabel)

![](assets/chinookschema2.png)

Gambar di atas adalah ERD untuk database chinook:

- Satu database terdiri dari beberapa tabel
- Setiap tabel terdiri dari beberapa atribut/kolom
- Terdapat kolom unik yang membedakan satu tabel dengan lainnya, disebut sebagai **Primary Key (PK)**. PK dapat digunakan sebagai kolom penghubung antar tabel saat dilakukan proses join (penggabungan) tabel.

## Database Connection

Sedikit berbeda dengan `pandas` yang semua operasinya dilakukan di komputer lokal Anda. Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan **terpusat** pada server dan dapat **diakses oleh beberapa client**. Berikut adalah ilustrasinya:

![](assets/clientserver.png)

Untuk menghubungkan client machine dengan server, kita perlu membuat sebuah connection melalui sebuah DBAPI. Pada Python, DBAPI ekuivalen dengan sebuah package yang memungkinkan seorang data analyst dapat membuat objek **database connection**, salah satunya adalah `sqlite3`. Berikut adalah ilustrasi dari DBAPI:

![](assets/dbapi.png)

**[OPSIONAL]** Berikut adalah contoh package lain pada Python untuk mengkoneksikan dengan database:

<br>

<details>
    <summary>✨ Connecting to MySQL</summary>
    
```python
import pymysql
  
conn = pymysql.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    db = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Oracle</summary>
    
```python
import cx_Oracle
  
# data source name from tnsnames.ora file
dsn_tns = cx_Oracle.makedsn(
    HOST_NAME,
    PORT_NUMBER
    service_name = SERVICE_NAME)

# connection
conn = cx_Oracle.connect(
    user = USER_NAME,
    password = PASSWORD,
    dsn = dsn_tns)
```
</details>

<br>

<details>
    <summary>✨ Connecting to PostgreSQL</summary>
    
```python
import psycopg2

conn = psycopg2.connect(
    host = HOST_NAME,
    port = PORT_NUMBER,
    user = USER_NAME,
    password = PASSWORD,
    database = DATABASE_NAME)
```
</details>

<br>

<details>
    <summary>✨ Connecting to Microsoft SQL Server</summary>
    
```python
import pyodbc 
conn = pyodbc.connect(
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=host;'
    'PORT=1433;'
    'UID=user;'
    'PWD=password;'
    'Database=database;')
```
</details>

Sebagai tahap awal, mari kita coba bagaimana mengkoneksikan Jupyter Notebook dengan database SQLite (menggunakan package `sqlite3`) yang disebut sebagai **connection**:

In [1]:
import sqlite3 # default package: Database API (DBAPI) / package
import pandas as pd

In [2]:
# membuat objek connection
conn = sqlite3.connect("./data_input/chinook.db")
conn

## `SELECT` Statements

`SELECT` digunakan untuk memilih **kolom** dari sebuah **tabel**.

Syntax `SELECT`:

```sql
SELECT <NAMA_KOLOM>
FROM <NAMA_TABLE>
```

Melalui `pandas` kita dapat menggunakan method method `pd.read_sql_query(sql, con)`. Parameter:

- `sql`: SQL query dalam bentuk string
- `con`: object connection

Misal kita hanya ingin mengambil kolom `AlbumId` dan `Title` dari tabel `albums`:

In [3]:
pd.read_sql_query('SELECT AlbumId, Title FROM albums', conn)

,AlbumId,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,3,Restless and Wild
3,4,Let There Be Rock
4,5,Big Ones
...,...,...
342,343,Respighi:Pines of Rome
343,344,Schubert: The Late String Quartets & String Qu...
344,345,Monteverdi: L'Orfeo
345,346,Mozart: Chamber Music


Gunakan command `SELECT *` untuk mengambil **semua kolom** dari tabel:

In [4]:
pd.read_sql_query('SELECT * from albums', conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


💡 **Note**: Berbeda dari Python, perintah yang ada di SQL bersifat case **insensitive**. Query di atas dapat juga dituliskan sebagai `select * from albums`. Namun, demi kemudahan pembacaan, statement biasa dituliskan dengan huruf besar (dalam hal ini `SELECT` dan `FROM`).

In [5]:
pd.read_sql_query('select * from albums', conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


### ❓ Knowledge Check

Kali ini, coba ambil semua kolom pada tabel `artists` kemudian simpan ke dalam sebuah object dataframe bernama `artist`.

**Pertanyaan:** Berapa baris yang ada pada dataframe `artist`?

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
```python
___ = pd.read_sql_query("SELECT ___ FROM ___", conn)
```
</details>

In [6]:
# your code here
artist = pd.read_sql_query('select * from artists', conn)
artist

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


Berikut adalah parameter yang sering digunakan dalam method `pd.read_sql_query()`:

- `sql`: SQL query dalam bentuk string
- `con`: object connection
- `index_col`: nama atau index kolom yang ingin dijadikan index (seperti pada `pd.read_csv()`)
- `parse_dates`: nama kolom yang ingin dikonversi menjadi tipe data `datetime64` (seperti pada `pd.read_csv()`)

> Kunjungi [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) untuk detail lebih lanjut.

### LIMIT

Statement `LIMIT` digunakan untuk mengambil beberapa baris pertama pada data.

Kita ingin ambil **5 baris pertama** dari tabel `artists` lalu set kolom `ArtistId` menjadi index.

In [7]:
pd.read_sql_query('select * from artists limit 5', index_col='ArtistId',con=conn)

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [8]:
pd.read_sql_query('PRAGMA table_info(albums)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,AlbumId,INTEGER,1,None,1
1,1,Title,NVARCHAR(160),1,None,0
2,2,ArtistId,INTEGER,1,None,0


## SQL Joins

Statement `JOIN` digunakan untuk **menggabungkan dua tabel** menjadi satu tabel melalui **kolom penghubung** yang sama. Operasi `JOIN` dapat dilakukan lebih dari satu kali dalam satu SQL query. Berikut adalah diagram ilustrasi beberapa macam operasi SQL `JOIN`:

![](assets/sqljoins.png)
Credit: Data & Object Factory, LLC

- `LEFT JOIN` paling sering digunakan pada sebagian besar skenario bisnis. `LEFT JOIN` mengembalikan semua baris pada tabel kiri **terlepas** dari apakah ada baris yang cocok pada tabel kanan.
- `INNER JOIN` adalah tipe join yang sangat intuitif dan mudah dipahami. Query ini mengembalikan semua baris di tabel kiri yang cocok dengan tabel kanan.

**Note**:

- `RIGHT JOIN` hampir tidak pernah digunakan karena ekuivalen dengan `LEFT JOIN` hanya mengganti peletakkan tabel kiri dan kanannya saja.
- `FULL OUTER JOIN` sangat jarang digunakan. Selain alasan komputasi, tipe join ini mengembalikan semua baris dari kedua tabel terlepas dari apakah ada kecocokan atau tidak, sehingga menghasilkan DataFrame dengan banyak missing value.

> Ilustrasi terkait ke-empat tipe join dapat dilihat pada [Visual JOIN](https://joins.spathon.com/).

Syntax `JOIN`:

```sql
SELECT <COLUMNS>
FROM <LEFT_TABLE>
[LEFT|INNER] JOIN <RIGHT_TABLE>
ON <LEFT_TABLE>.key = <RIGHT_TABLE>.key
```

**Kasus:** Kita ingin mendapatkan DataFrame yang menampilkan semua baris dengan kolom `AlbumId`, `Title`, dan `Name`. 

Perhatikan bahwa kolom `Name` terdapat pada tabel `artists`, sedangkan kolom `AlbumId` dan `Title` terdapat pada tabel `albums`. Berikut skema pada database `chinook.db`:

1. Tabel `albums`: 
    - `AlbumId`
    - `Title`
    - `ArtistId`


2. Tabel `artists`:
    - `ArtistId`
    - `Name` 

In [9]:
albums = pd.read_sql_query("SELECT AlbumId, Title, Name \
                            FROM albums \
                            LEFT JOIN artists \
                            ON artists.ArtistId = albums.ArtistId", conn)
albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


Perhatikan SQL query di atas:
- Tabel `albums` menjadi **tabel kiri**, sedangkan tabel `artists` menjadi **tabel kanan**.
- Kedua tabel digabungkan menggunakan operasi `LEFT JOIN` melalui key `ArtistId`.
- Penggunaan karakter backslash (`\`) ditujukan agar ke-empat baris SQL query dibaca sebagai satu kesatuan baris. Namun penggunaan backslash kurang efisien dan memiliki resiko error yang lebih tinggi. **Sebagai alternatif, Anda dapat mengapit SQL query dengan petik tiga kali `"""` atau `'''`**

In [10]:
# re-create tabel di atas menggunakan petik tiga kali 
albums = pd.read_sql_query("""SELECT AlbumId, Title, Name 
                            FROM albums 
                            LEFT JOIN artists 
                            ON artists.ArtistId = albums.ArtistId""", conn)
albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


Terkadang penamaan kolom ataupun tabel pada database cukup panjang dan redundan untuk diketik, ataupun belum cukup informatif.

Penggunaan statement `AS` dapat digunakan untuk melakukan **aliasing / rename** nama tabel maupun nama kolom.

In [11]:
# re-create tabel di atas menggunakan aliasing
albums = pd.read_sql_query("""SELECT AlbumId, Title, Name 
                            FROM albums AL
                            LEFT JOIN artists AR
                            ON AL.ArtistId = AR.ArtistId""", conn)
albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


> ⚠️ Hati-hati! Saat melakukan `LEFT JOIN`, peletakkan tabel kiri dan kanan sangat berpengaruh terhadap hasil penggabungan.

Sebagai perbanding, sekarang coba kita tukar tabel `artists` menjadi tabel kiri, sedangkan `albums` menjadi tabel kanan.

Object `x` di bawah menampilkan semua data `artists` walaupun artist tersebut tidak ada di tabel `albums`.

In [12]:
x = pd.read_sql_query(
    """
    SELECT AlbumId, Title, Name
    FROM artists
    LEFT JOIN albums
    ON artists.ArtistId = albums.ArtistId
    """, conn)

# cek baris dengan missing value
x[x.isna().any(axis=1)]

,AlbumId,Title,Name
50,NaN,None,Milton Nascimento & Bebeto
51,NaN,None,Azymuth
55,NaN,None,João Gilberto
56,NaN,None,Bebel Gilberto
57,NaN,None,Jorge Vercilo
...,...,...,...
327,NaN,None,DJ Dolores & Orchestra Santa Massa
328,NaN,None,Seu Jorge
329,NaN,None,Sabotage E Instituto
330,NaN,None,Stereo Maracana


### ❓ Knowledge Check

Buatlah sebuah object dataframe bernama `tracks` yang menampilkan:

- Semua kolom pada tabel `tracks`
- Kolom `Title` pada tabel `albums`
- Kolom `Name` pada tabel `artists`
- Kolom `Name` pada tabel `genres`

Lalu set kolom `TrackId` menjadi index pada dataframe tersebut. Apabila sudah benar, seharusnya dataframe Anda memiliki 3503 baris dan 11 kolom.

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

> **Hint 1**: Pada kasus ini, Anda wajib menggunakan syntax `<nama_tabel>.<nama_kolom>` pada statement `SELECT`. Hal ini karena terdapat nama kolom yang sama dari tabel yang berbeda. Jangan lupa untuk menggunakan aliasing `AS` agar nama kolom akhir tidak ambigu.
    
> **Hint 2**: Anda dapat menggunakan `SELECT tracks.*` untuk mengambil semua kolom pada tabel `tracks`

> **Hint 3**: Anda dapat melanjutkan hasil operasi `LEFT JOIN` dengan `LEFT JOIN` berikutnya. Sebagai referensi, berikut adalah struktur kodenya:
    
```python
___ = pd.read_sql_query(
    """
    SELECT ___
    FROM ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    LEFT JOIN ___
    ON ___
    """, conn, index_col=___)
```
</details>

In [13]:
pd.read_sql_query('PRAGMA table_info(tracks)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


In [14]:
# your code here
tracks = pd.read_sql_query(sql="""
                    SELECT t.*,
                    al.title, ar.name AS ArtistName, g.name as GenreName from tracks t
                    LEFT JOIN albums al ON al.albumId = t.albumId
                    LEFT JOIN artists ar ON ar.artistId = al.artistId
                    LEFT JOIN genres g ON g.genreId = t.genreId
                    """, con=conn, index_col='TrackId')
tracks.shape

(3503, 11)

Gunakan object dataframe `tracks` yang telah Anda peroleh di atas dalam menjawab pertanyaan berikut:

1. Pilihlah semua nama genre yang terdapat pada 5 baris terakhir pada dataframe `tracks`. Jawaban bisa lebih dari satu.
    - [ ] Latin
    - [X] Classical
    - [X] Soundtrack
    - [ ] Pop

In [15]:
# your code here
tracks.tail(5)

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,ArtistName,GenreName
TrackId,,,,,,,,,,,
3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Respighi:Pines of Rome,Eugene Ormandy,Classical
3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Classical
3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Classical
3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,Mozart: Chamber Music,Nash Ensemble,Classical
3503,Koyaanisqatsi,347,2,10,Philip Glass,206005,3305164,0.99,Koyaanisqatsi (Soundtrack from the Motion Pict...,Philip Glass Ensemble,Soundtrack


2. Di antara genre berikut, manakah yang termasuk dalam top 3 genre yang paling banyak muncul pada dataframe `tracks`? Buatlah sebuah tabel frekuensi dengan metode yang telah Anda pelajari!

    - [X] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>

Beberapa pilihan method yang dapat Anda gunakan:
    
- `.value_counts()`
- `pd.crosstab(index=___, columns=___)`
- `.pivot_table(index=___, aggfunc='count')`
- `.groupby().count()`
</details>

In [16]:
# your code here
tracks['GenreName'].value_counts().sort_values()

Opera                    1
Rock And Roll           12
Science Fiction         13
Bossa Nova              15
Comedy                  17
Easy Listening          24
Sci Fi & Fantasy        26
Heavy Metal             28
World                   28
Electronica/Dance       30
Hip Hop/Rap             35
Alternative             40
Soundtrack              43
Pop                     48
Reggae                  58
R&B/Soul                61
Drama                   64
Classical               74
Blues                   81
TV Shows                93
Jazz                   130
Alternative & Punk     332
Metal                  374
Latin                  579
Rock                  1297
Name: GenreName, dtype: int64

3. Gunakan `groupby()` pada kolom `ArtistName` untuk menghitung `mean()` dari `UnitPrice`. Anda akan melihat bahwa mayoritas artists memiliki rata-rata `UnitPrice` di angka 0.99, namun ada juga beberapa artists yang rata-ratanya 1.99. Dari nama artists berikut, pilihlah yang memiliki rata-rata `UnitPrice` sebesar 0.99:

    - [ ] The Office
    - [ ] Aquaman
    - [X] Pearl Jam
    - [ ] Lost

In [17]:
# your code here
tracks_artist = tracks.pivot_table(
    index='ArtistName',
    values='UnitPrice',
    aggfunc='mean'
)
tracks_artist.loc[['The Office','Aquaman','Pearl Jam', 'Lost']]

,UnitPrice
ArtistName,
The Office,1.99
Aquaman,1.99
Pearl Jam,0.99
Lost,1.99


## SQL Aggregation

Di `pandas`, kita bisa menggunakan ketiga method berikut untuk membuat tabel agregasi:

- `.crosstab()`
- `.pivot_table()`
- `.groupby()`

Sedangkan di SQL, kita menggunakan statement `GROUP BY` yang diletakkan setelah `SELECT ... FROM ...`.

Misal kita ingin mengetahui top 5 `CustomerId` berdasarkan jumlah besaran transaksinya (`Total`) dan juga tampilkan banyak transaksinya:

In [18]:
top_cust = pd.read_sql_query("SELECT CustomerId, SUM(Total) AS TotalValue, \
                              COUNT(InvoiceId) AS Purchases \
                              FROM invoices \
                              GROUP BY CustomerId \
                              ORDER BY TotalValue DESC \
                              LIMIT 5", conn, index_col='CustomerId')
top_cust

,TotalValue,Purchases
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


In [19]:
# re-create the table above
top_cust = pd.read_sql_query("""SELECT CustomerId, SUM(Total) AS TotalValue, 
                              COUNT(InvoiceId) AS Purchases 
                              FROM invoices 
                              GROUP BY CustomerId 
                              ORDER BY TotalValue DESC 
                              LIMIT 5""", conn, index_col='CustomerId')
top_cust

,TotalValue,Purchases
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


Perhatikan bagaimana query di atas dapat mengambil 5 customer teratas dari tabel `invoices`:

1. Data dikelompokkan berdasarkan `CustomerId` dengan statement `GROUP BY`
2. Terdapat dua fungsi agregasi yang digunakan: `SUM()` dan `COUNT()`, masing-masing mengagregasikan kolom `Total` dan `InvoiceId`. Berikut adalah fungsi agregasi yang sering digunakan: `SUM`,` AVG`, `COUNT`, `MIN`, dan `MAX`.
3. Statement `ORDER BY` ditambahkan untuk mengurutkan tabel berdasarkan kolom `TotalValue` secara `DESC`.

💡 **Note**:

- SQL Statement `GROUP BY` ekuivalen dengan `.groupby()` pada `pandas`
- SQL Statement `ORDER BY` ekuivalen dengan `.sort_values()` pada `pandas`

### ❓ Knowledge Check

Seorang produser musik ingin menciptakan lagu dengan genre yang akan laku di market. Sebelum mulai, ia terlebih dahulu melakukan riset tentang genre musik apa yang sedang hits saat ini. Sebagai seorang data analyst, Anda diminta untuk membantu produser musik dalam membuat keputusan yang tepat.

**Pertanyaan:** Tampilkan top 5 genre musik yang memiliki jumlah penjualan terbanyak, sertakan juga dengan total quantity yang terjual.

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
Berikut adalah code yang dapat Anda gunakan untuk mengetahui `genres` paling populer dari semua invoice sales.

```python
top_genre = pd.read_sql_query(
    """
    SELECT
    genres.GenreId,
    genres.Name,
    ___(invoices.Total),
    ___(invoice_items.Quantity)
    FROM invoices
    LEFT JOIN ___ ON ___
    LEFT JOIN ___ ON ___
    LEFT JOIN ___ ON ___
    GROUP BY ___
    ORDER BY ___
    """,
    conn,
    index_col='GenreId'
```
</details>

In [20]:
pd.read_sql_query(sql="""
                    SELECT g.GenreId, g.Name,
                    SUM(inv.Total) as TotalPenjualan,
                    SUM(ii.Quantity) as TotalKuantitas
                    FROM invoices inv
                    LEFT JOIN invoice_items ii ON ii.invoiceLineId = inv.invoiceId
                    LEFT JOIN tracks t ON t.trackId = ii.trackId
                    LEFT JOIN genres g ON g.genreId = t.genreID
                    GROUP BY g.genreId
                    ORDER BY 3 DESC
                    LIMIT 5
                    """, con=conn)

,GenreId,Name,TotalPenjualan,TotalKuantitas
0,1,Rock,842.01,160
1,7,Latin,419.83,81
2,3,Metal,366.74,56
3,4,Alternative & Punk,353.43,56
4,2,Jazz,132.75,19


💡 Panduan step-by-step:

1. Lakukan operasi `JOIN` tergantung dari kebutuhan kolom yang ingin dianalisis
2. Gunakan `GROUP BY` apabila ingin membuat tabel agregasi
3. Aplikasikan fungsi agregat di `SELECT`
4. Tambahkan kolom yang relevan untuk ditampilkan pada `SELECT`
5. Urutkan data menggunakan `ORDER BY`

## `WHERE` Statements

Sampai di sini, kita telah mempelajari beberapa statement SQL yang sering digunakan:

- `SELECT` statement
- SQL `JOIN`
- Aliasing
- SQL Aggregation dengan `GROUP BY`

Sekarang, kita akan melihat teknik untuk melakukan **conditional subsetting atau filter baris** menggunakan statement `WHERE` yang diikuti dengan **kondisi**.

### Logical Operator

- Kondisi `WHERE` dapat dikombinasikan dengan logical operator: `IS`, `AND`, `OR`, dan `NOT`:
    - `IS` sama saja seperti notasi matematis `=`
    - `IS NOT` seperti notasi `!=`
    
- Kondisi pada `WHERE` juga mendukung operator matematis seperti >, >=, <, dan <=

**Kasus**: kita ingin melakukan analisis terhadap semua data `invoices` yang terjadi di `BillingCountry` Germany, maka kita bisa menambahkan statement `WHERE` sebagai berikut:

In [21]:
pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE BillingCountry = 'Germany'
    """,
    conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
5,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
6,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
7,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
8,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


Berlawanan dengan kasus di atas, misal kita ingin analisis semua data `invoices` dimana `BillingCountry` **selain** negara Germany:

In [22]:
pd.read_sql_query(
"""
SELECT *
FROM invoices
WHERE BillingCountry != 'Germany'
""",
conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
379,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
380,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
381,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
382,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


**Kasus**: Misalkan kita hanya tertarik untuk menganalisis data yang terjadi di Amerika (yaitu `BillingCountry` Canada dan USA). Kira-kira bagaimana cara mengimplementasikannya dalam bentuk query SQL?

In [23]:
# your code here
pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE BillingCountry = 'Canada'OR BillingCountry = 'USA'
    """,
conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
...,...,...,...,...,...,...,...,...,...
142,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
143,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
144,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
145,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


### `IN` Operator

Operator `IN` yang memungkinkan kita menentukan beberapa nilai untuk perbandingan. Misalnya, seperti pada kasus sebelumnya, kita ingin mengambil semua data `invoices` yang terjadi di negara (`BillingCountry`) Canada dan juga USA:

In [24]:
north_america = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE BillingCountry IN ('Canada', 'USA')
    """,
    conn)

north_america.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98


### ❓ Knowledge Check


Ambillah seluruh data musik yang memiliki genre `Pop` dan `UnitPrice` dari tracknya adalah 0.99. Kemudian simpan ke object dataframe bernama `popmusic`.

**Pertanyaan:** Ada berapa baris pada dataframe `popmusic`?

<br>

<details>
    <summary><i>👉 Klik di sini untuk hint</i></summary>
    
```python
popmusic = pd.read_sql_query(
    """
    SELECT tracks.*, genres.Name AS GenreName
    FROM ___
    LEFT JOIN ___
    ON ___
    WHERE ___
    """,
    conn,
    index_col='TrackId'
)
```
</details>

In [25]:
# your code here
popmusic = pd.read_sql_query(sql="""
                SELECT t.*, g.Name AS GenreName
                FROM tracks t
                LEFT JOIN genres g ON g.GenreId = t.GenreId
                WHERE t.UnitPrice = 0.99 AND g.Name = 'Pop'
                """, con=conn, index_col='TrackId')
popmusic.shape

(48, 9)

In [26]:
pd.read_sql_query('''
                SELECT AVG(t.Milliseconds) AS RataDurasi
                FROM tracks t
                LEFT JOIN genres g ON g.genreId = t.genreId
                WHERE g.name = 'Rock'
                ''', con=conn)

,RataDurasi
0,283910.043177


### Operating Dates

Pada operasi kondisi pada statement `WHERE` sebelumnya, kita dapat mengambil semua data `invoices` pada negara Germany. Namun, juga umum untuk kita melakukan conditional query untuk mengambil data pada **rentang tanggal** tertentu.

Sebelum lanjut, mari kita lihat tipe data dari objek `germany` yang telah diperoleh sebelumnya:

In [27]:
germany = pd.read_sql_query("SELECT * FROM invoices WHERE BillingCountry = 'Germany'", conn)
germany.dtypes

InvoiceId              int64
CustomerId             int64
InvoiceDate           object
BillingAddress        object
BillingCity           object
BillingState          object
BillingCountry        object
BillingPostalCode     object
Total                float64
dtype: object

Perhatikan bahwa `InvoiceDate` dikenali sebagai tipe data `object`. **Method `pd.read_sql_query()` berperilaku seperti method `pd.read_csv()` dimana secara default membaca tipe data suatu kolom sebagai numerik dan objek.** Ini tidak berarti bahwa kolom tersebut disimpan menggunakan format string (umumnya dikenal sebagai `VARCHAR` dalam database SQL). Lihatlah skema tabel berikut:

In [28]:
invoices_table = pd.read_sql_query(
    """
    SELECT sql
    FROM sqlite_master
    WHERE name = 'invoices'
    """, conn)
print(invoices_table.loc[0,:].values[0])

CREATE TABLE "invoices"
(
    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY ([CustomerId]) REFERENCES "customers" ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


> Perlu diingat bahwa untuk DBMS yang berbeda-beda, maka cara mengambil skema tabel akan berbeda pula. Query di atas khusus untuk mengambil skema tabel dari database SQLite (`sqlite_master`).

Output dari query di atas termasuk dalam tipe **Data Definition Language (DDL)** yang digunakan untuk membuat tabel. Dengan membaca DDL, akan berguna untuk memahami skema tabel dari database sehingga kita dapat melakukan operasi yang sesuai. Pada skema tabel `invoices` terdapat informasi yang berguna seperti:

- `InvoiceId` sebagai primary key
- `InvoiceDate` disimpan sebagai tipe data `DATETIME` (format `YYYY-MM-DD HH:MM:SS`)
- `CustomerId` sebagai foreign key pada tabel `customers`

Jika Anda tidak disediakan dengan skema database dalam bentuk diagram, maka luangkan waktu untuk mempelajari setiap skema tabel melalui DDL-nya.

**Kasus**: Kita ingin meninjau penjualan (`invoices`) tahun lalu di seluruh negara pada tahun 2012 saja.

In [29]:
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE strftime('%Y', InvoiceDate) = '2012'
    """,
    conn,
    parse_dates='InvoiceDate'
)
invoice_2012

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,250,55,2012-01-01,421 Bourke Street,Sidney,NSW,Australia,2010,13.86
1,251,10,2012-01-09,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,0.99
2,252,11,2012-01-22,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,1.98
3,253,13,2012-01-22,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,1.98
4,254,15,2012-01-23,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,3.96
...,...,...,...,...,...,...,...,...,...
78,328,15,2012-12-15,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,0.99
79,329,16,2012-12-28,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,1.98
80,330,18,2012-12-28,627 Broadway,New York,NY,USA,10012-2612,1.98
81,331,20,2012-12-29,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,3.96


### `BETWEEN` Operator

Melanjutkan kasus sebelumnya, apabila kita ingin menggunakan kondisi pada rentang tertentu, pendekatan yang umum digunakan adalah operator `BETWEEN`. Silahkan lengkapi code berikut dan lihat apakah data yang terambil sama seperti sebelumnya:

```python
invoice_2012 = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE ___ BETWEEN '___' AND '___'
    """,
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()
```

In [30]:
# your code here
invoice_2012 = pd.read_sql_query('''
                SELECT *
                FROM invoices
                ''', con=conn, parse_dates='InvoiceDate')

⚠️ **Hati-hati!**

Operator `BETWEEN` sejatinya adalah inclusive, di mana kondisi start dan endnya termasuk. Namun, saat kita membandingkan date (misal '2012-12-31') dengan datetime (pada kolom `InvoiceDate`), seakan-akan end tidak inclusive. Sebagai eksperimen, cobalah ganti kondisi pada statement `WHERE` menjadi:

```
WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
```

Maka `InvoiceDate` pada tanggal `2012-12-30` tidak masuk.

Sebagai solusinya, lebih baik kita menambahkan komponen waktu (time) pada kondisi:
```
WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 00:00:00'
```

In [31]:
# bukti bahwa operator BETWEEN inclusive pada start dan end
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE InvoiceId BETWEEN 2 AND 5
    """,
    conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [32]:
# MASALAH: ketika hanya menggunakan '2012-12-30' maka data tanggal 30 Desember 2012 TIDAK masuk
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
    """,
    conn,
    parse_dates='InvoiceDate'
)['InvoiceDate'].describe()

/tmp/ipykernel_58017/3211621897.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pd.read_sql_query(


count                      82
unique                     70
top       2012-02-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-29 00:00:00
Name: InvoiceDate, dtype: object

In [33]:
# SOLUSI: masukkan komponen TIME (00:00:00) agar data transaksi pada tanggal 30 Desember 2012 juga termasuk
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-31 23:59:59'
    """,
    conn,
    parse_dates='InvoiceDate'
)['InvoiceDate'].describe()

/tmp/ipykernel_58017/548660248.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pd.read_sql_query(


count                      83
unique                     71
top       2012-01-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

> **Key takeaway:** Ketika ingin membuat sebuah kondisi `NAMA_KOLOM = NILAI`, pastikan `NILAI` memiliki tipe data yang sama seperti `NAMA_KOLOM`

### `LIKE` Operator

Operator `LIKE` sangat berguna jika kita perlu mencocokan bagian tertentu dari sebuah string daripada menggunakan operator sama dengan (`=`). `'107%'` yang Anda lihat dalam query ditujukan untuk mengekstrak nilai `BillingPostalCode` yang **dimulai** dengan angka 107. Ini sangat membantu ketika Anda ingin mengekstrak data hanya pada wilayah tertentu. Di negara Germany, kita akan tahu bahwa Wilmersdorf dan Tempelhof di Berlin memiliki kode pos dimulai dengan 107.

> **Note:** Karakter `%` disebut juga sebagai [wildcard character](https://www.w3schools.com/sql/sql_wildcards.asp)

In [34]:
pd.read_sql_query(
    """
    SELECT * FROM invoices
    WHERE BillingCountry = 'Germany'
    AND BillingPostalCode LIKE '107%'
    """,
    conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


**Diskusi:**

- Jika `BillingPostalCode LIKE '107%'`, maka semua baris dengan kode pos **dimulai** angka 107 akan tampil.
- Jika `BillingPostalCode LIKE '%107'`, maka semua baris dengan kode pos **diakhiri** angka 107 akan tampil.

Menurut Anda apa yang akan muncul jika kita menggunakan `%` **sebelum dan sesudah** pola, yaitu `BillingPostalCode LIKE '%107%'`

#### ❓ Knowledge Check

**Kasus**: Pada 5 data pertama `customerinv`, kita bisa melihat kolom `Company` mungkin tidak dapat diandalkan karena bernilai `None`. Tetapi jika Anda memperhatikan kolom `Email`, Anda dapat melihat beberapa customer memiliki domain email `apple`, yang bisa menjadi indikator perusahaan mereka.

In [35]:
customerinv = pd.read_sql_query(
    """
    SELECT FirstName, LastName, Email, Company,
    InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    """,
    conn)

customerinv.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


**Pertanyaan:** Berapa pelanggan yang bekerja di Apple Inc., jika kita menggunakan domain `Email` sebagai pengganti indikator `Company`?

- [ ] 412
- [X] 49
- [ ] 7
- [ ] 14

Silahkan lengkapi kode berikut:

```python
applecust = pd.read_sql_query(
    """
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices.___ = customers.___
    WHERE ___
    """,
    conn
)
```

In [36]:
# your code here
applecust = pd.read_sql_query(
    """
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    WHERE customers.Email LIKE '%apple%'
    """,
    conn
)
applecust.shape

(49, 8)

## 📝 Summary

### ❓ Knowledge Check 1: SQL Statement Function

Kita telah mempelajari banyak statement pada SQL, silahkan mencocokan statement sesuai dengan kegunaannya:

**KEGUNAAN**

A. Memberikan nama lain pada tabel maupun kolom

B. Mengambil beberapa baris teratas dari tabel

C. Mengurutkan baris berdasarkan nilai pada kolom

D. Mengambil kolom dari sebuah tabel

E. Menggabungkan dua tabel menjadi satu tabel berdasarkan kolom penghubung

F. Filter baris

G. Membuat tabel agregasi

**STATEMENT**

1. `SELECT <nama_kolom> FROM <nama_tabel>`

2. `LIMIT <banyaknya_baris>`

3. `AS <nama_kolom_atau_baris>`

4. `<tabel_kiri> [LEFT|INNER] JOIN <tabel_kanan> ON <tabel>.key = <tabel>.key`

5. `GROUP BY <nama_kolom>`

6. `ORDER BY <nama_kolom> [ASC|DESC]`

7. `WHERE <kondisi>`

**JAWABAN**

1. D

2. B

3. A

4. E

5. G

6. C

7. F

### ❓ Knowledge Check 2: SQL Statement Structure

Susunlah 8 baris SQL statement berikut menjadi sebuah kerangka urutan syntax yang benar:

1. `GROUP BY <nama_kolom>`

2. `LIMIT <banyaknya_baris>`

3. `SELECT <nama_kolom> AS ...`

4. `[LEFT|INNER] JOIN <tabel_kanan> AS ...`

5. `ORDER BY <nama_kolom> [ASC|DESC]`

6. `FROM <nama_tabel> AS ...`

7. `WHERE <kondisi>`
    
8. `ON <tabel>.key = <tabel>.key`

> **JAWABAN**: Urutan yang benar adalah ...

- 

### 💭 Dive Deeper

Toko musik digital ini ingin memberikan satu penghargaan ke `employees` yang telah bekerja keras melakukan penjualan di `BillingCountry` Amerika Selatan (yaitu negara **Argentina, Brazil, dan Chile**). Apabila penghargaan tersebut diberikan kepada `employees` berdasarkan **jumlah `Total`** penjualan, siapakah yang berhak mendapatkannya? Tampilkan **nama lengkap (`FirstName` dan `LastName`) beserta total penjualannya**.

In [37]:
# pd.read_sql_query('Select * from employees e JOIN customers c ON c.SupportRepId = e.EmployeeId', conn)

In [38]:
# your code here
pd.read_sql_query(sql='''
                    SELECT e.FirstName, e.LastName,
                    SUM(inv.Total) AS TotalPenjualan
                    FROM employees e
                    LEFT JOIN customers c ON c.SupportRepId = e.EmployeeId
                    LEFT JOIN invoices inv ON c.CustomerId = inv.CustomerId
                    WHERE inv.BillingCountry IN ('Argentina', 'Brazil', 'Chile')
                    GROUP BY e.EmployeeId
                    ORDER BY 3 DESC
                    ''', con=conn)

,FirstName,LastName,TotalPenjualan
0,Margaret,Park,112.86
1,Steve,Johnson,84.24
2,Jane,Peacock,77.24


# Under and Over Fetching

Konsep client-server pada database: Sedikit berbeda dengan `pandas` yang semua operasinya dilakukan di komputer lokal Anda. Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan terpusat pada server dan dapat diakses oleh beberapa client.

![](assets/clientserver.png)

Saat Anda melakukan query, sebenarnya Anda sedang menjalankan perintah untuk mengunduh data ke komputer lokal. Proses pengunduhan ini membutuhkan sumber daya dan perlu memanfaatkan alat secara efektif untuk meminimalkan biaya.

- **Over** fetching adalah kondisi dimana kita menarik data dari database **lebih dari** yang dibutuhkan, sehingga membutuhkan biaya dan waktu yang lebih.

- **Under** fetching adalah kondisi sebaliknya, dimana data yang ditarik **kurang dari** yang dibutuhkan, sehingga proses analisis tidak dapat dilakukan secara lengkap.

**Diskusi:**

Anda diminta untuk melakukan analisis terhadap semua penjualan (`invoices`) genre `Rock` pada tahun 2012. Pertimbangkan pertanyaan berikut:

- Apakah perlu bagi Anda untuk mengunduh semua tabel `tracks` ke komputer lokal? => Tidak
- Apakah Anda melakukan filter baris terhadap `tracks` dengan genre `Rock` menggunakan SQL statement `WHERE` atau conditional subsetting `pandas`? => Lebih baik menggunakan `WHERE` nya SQL, agar tidak overfetching
- Karena kita memerlukan informasi dari beberapa tabel, manakah cara yang lebih nyaman: melakukan query dengan `JOIN` atau melakukan `SELECT` secara terpisah dari database? => `JOIN`, tapi jangan lupa untuk memilih kolom yang cukup dipakai saja, kalau tidak nanti semua kolom dari setiap tabel yang dijoin akan diikut sertakan.

Cobalah membuat query yang paling optimum menurut Anda:

In [39]:
# your code here
pd.read_sql_query(sql='''
                    SELECT inv.*, g.name
                    FROM invoices inv
                    LEFT JOIN invoice_items ii ON ii.invoiceLineId = inv.invoiceId
                    LEFT JOIN tracks t ON t.trackId = ii.trackId
                    LEFT JOIN genres g ON g.genreId = t.genreId
                    WHERE g.name='Rock'
                    ''', con=conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,Name
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,Rock
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,Rock
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,Rock
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,Rock
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86,Rock
...,...,...,...,...,...,...,...,...,...,...
155,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86,Rock
156,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99,Rock
157,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98,Rock
158,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98,Rock


# (Optional) SQL Subquery

Dalam beberapa kasus tertentu, kita ingin melakukan filter baris berdasarkan syarat tertentu dimana nilai-nilai kondisi didapatkan dari hasil query lain. Apakah Anda masih ingat bagaimana kita mengambil semua pelanggan yang memiliki `invoices` dengan Total paling tinggi? (Bagian SQL Aggregation)

In [40]:
# menampilkan top 5 customers dengan total pembelanjaan terbanyak
pd.read_sql_query(
    """
    SELECT CustomerId, SUM(Total) AS TotalValue,
    COUNT(InvoiceId) AS Purchases
    FROM invoices
    GROUP BY CustomerId
    ORDER BY TotalValue DESC
    LIMIT 5
    """,
    conn,
    index_col='CustomerId'
)

,TotalValue,Purchases
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


Misalnya dari tabel tersebut kita ingin menarik semua data `invoices` berdasarkan top 5 `customers`. Untuk melakukan hal tersebut, kita akan menggunakan subquery setelah statement `WHERE` menggunakan operator `IN`. Sebenarnya kita bisa saja menuliskan list `CustomerId` dalam kondisi secara **hard-code** seperti ini:

In [41]:
# menampilkan invoices hanya untuk top 5 customers
top5cust_hardcode = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE CustomerId IN (6, 26, 57, 45, 46)
    """,
    conn)

top5cust_hardcode['CustomerId'].unique()

array([ 6, 26, 45, 46, 57])

Namun cara hard-code seperti itu tidak disarankan. Bagaimana kalau database kita terus bertambah, sehingga bukan lagi `CustomerId` 6, 26, 57, 45, 46 yang merupakan top 5 customers? Maka dari itu, kita perlu menggunakan subquery agar **query kita lebih dinamis terhadap perubahan data**.

Untuk itu mari kita persiapkan **subquery** yang hanya mengembalikan list Top 5 `CustomerId` berdasarkan `Total` pembelian:

In [42]:
# subquery
pd.read_sql_query(
    """
    SELECT CustomerId
    FROM invoices
    GROUP BY CustomerId
    ORDER BY SUM(Total) DESC
    LIMIT 5
    """,
    conn)

,CustomerId
0,6
1,26
2,57
3,45
4,46


Subquery di atas kita gunakan untuk menggantikan query yang hard-code:

In [43]:
top5cust_subquery = pd.read_sql_query(
    """
    SELECT *
    FROM invoices
    WHERE CustomerId IN (
        SELECT CustomerId
        FROM invoices
        GROUP BY CustomerId
        ORDER BY SUM(Total) DESC
        LIMIT 5
    )
    """,
    conn)

top5cust_subquery

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,46,6,2009-07-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,8.91
1,175,6,2011-02-15 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
2,198,6,2011-05-20 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,3.96
3,220,6,2011-08-22 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,5.94
4,272,6,2012-04-11 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,0.99
5,393,6,2013-10-03 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,1.98
6,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86
7,70,26,2009-11-07 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,1.98
8,93,26,2010-02-09 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,3.96
9,115,26,2010-05-14 00:00:00,2211 W Berry Street,Fort Worth,TX,USA,76110,5.94


Memang hasil `top5cust_hardcode` dengan `top5cust_subquery` akan sama persis, namun secara sintaks `top5cust_subquery` jauh lebih baik karena code lebih dinamis.

In [44]:
top5cust_hardcode.shape

(35, 9)

In [45]:
# keuntungan: code lebih dinamis kalau ada perubahan data di masa depan
top5cust_subquery.shape

(35, 9)